CREATION D'UN WVAP 

Situation: Généralement lorqu'on est une grande institution qui opère des opérations financères et qu'on souhaite rentrer en position longue sur un actif il est important de bien se positioner à l'entrée

   PROBLEME:  la plus part des acteurs font face à un dilemne qui consite à se positionner au bon moment et à un prix raisonnable

SOLUTION:. Dès lors pour réduire le risque et rentrer à un prix raisonnable il est important d'utiliser un wvap qui va nous permettre d'avoir un bref aperçu de la tendance de marché le but étant d'eviter de rentrer en période de surévaluation de l'actif.

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
data=pd.read_csv('Ethereum Historical Data.csv')
data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,01/01/2025,"3,360.14","3,337.80","3,373.11","3,315.98",209.22K,0.67%
1,12/31/2024,"3,337.80","3,361.99","3,449.27","3,317.39",333.44K,-0.72%
2,12/30/2024,"3,361.99","3,355.97","3,436.83","3,306.11",494.99K,0.18%
3,12/29/2024,"3,355.97","3,404.53","3,412.32","3,327.45",176.43K,-1.43%
4,12/28/2024,"3,404.59","3,334.23","3,427.65","3,322.42",196.38K,2.11%


In [53]:
del data['High']
del data['Low']
del data['Change %']
del data['Open']

In [54]:
data.head()

,Date,Price,Vol.
0,01/01/2025,"3,360.14",209.22K
1,12/31/2024,"3,337.80",333.44K
2,12/30/2024,"3,361.99",494.99K
3,12/29/2024,"3,355.97",176.43K
4,12/28/2024,"3,404.59",196.38K


In [55]:
#convertir nos données volume en 'float'

In [56]:
def convert_volume_to_float(volume_str):
    if isinstance(volume_str,(int,float)):
        return volume_str
    Volume_str=str(volume_str).upper().strip() #mettre en majuscule et enlever les espaces
    if 'M' in volume_str:
        return float(Volume_str.replace('M',''))*1000000#millions
    elif'K' in volume_str:
        return float(volume_str.replace('K',''))*1000#milliers
    elif'B' in volume_str:
        return float(volume_str.replace('B',''))*1000000000#données en milliards
    else:
        try:#converti driectement si aucun suffixe n'est touvé directement
            return float(volume_str)
        except ValueError:
            return np.nan #remplace les valeurs vides par Nan

In [57]:
#appliquons la conversion 
data['Vol.']=data['Vol.'].apply(convert_volume_to_float)

In [58]:
print(data['Vol.'].dtype)

float64


In [60]:
#conversion de la colonne Price en entier
data['Price']=data['Price'].str.replace(',','.',regex=True)
data['Price']=data['Price'].str.replace('.','',regex=False)
data['Price']=data['Price'].astype(float)
#regex=false signifie que l'argument est traité comme un caractère littéral et non comme une expression régulière

In [61]:
print(data['Price'].dtype)

float64


In [68]:
#calcul de vwap

data['P*Q']=data['Price']*data['Vol.']
data['cumul_P*Q']=data['P*Q'].cumsum()
data['cumul_Vol.']=data['Vol.'].cumsum()
data['Vwap']=data['cumul_P*Q']/data['cumul_Vol.']

In [69]:
data.head()

,Date,Price,Vol.,P*Q,cumul_P*Q,cumul_Vol.,Vwap
0,01/01/2025,336014.0,209220.0,7.030085e+10,7.030085e+10,209220.0,336014.000000
1,12/31/2024,333780.0,333440.0,1.112956e+11,1.815965e+11,542660.0,334641.308149
2,12/30/2024,336199.0,494990.0,1.664151e+11,3.480116e+11,1037650.0,335384.373623
3,12/29/2024,335597.0,176430.0,5.920938e+10,4.072210e+11,1214080.0,335415.272470
4,12/28/2024,340459.0,196380.0,6.685934e+10,4.740803e+11,1410460.0,336117.516569


In [78]:

budget=10000000
positions=[]
#Fonction d'achat
for i, row in data.iterrows():
    Price=row['Price']
    vwap=row['Vwap']
    
    if Price< vwap and budget> 0:
        qty=budget// Price #nbr d'unité achetées
        if qty>0: #on s'assure qu'on peut bien acheter au moins une unité de l'actif
            cost=qty*Price
            budget-= cost#dans ce cas on procède à l'achat et le budget diminue
            positions.append((i,Price,qty,cost))
print('==Ordres Exécutés==')
for P in positions:
    print(f'{P[0]}| achat de {P[2]} unités à {P[1]} (coût={P[3]}) ')
print(f"\nBudget restant:{budget}")

==Ordres Exécutés==
1| achat de 29.0 unités à 333780.0 (coût=9679620.0) 
42| achat de 1.0 unités à 306972.0 (coût=306972.0) 
1737| achat de 1.0 unités à 13286.0 (coût=13286.0) 

Budget restant:122.0
